In [5]:
import os
from pathlib import Path
import scipy.io as sio

def load_drivecycle_mat_files(base_folder: str):
    """
    Scans the 'drivecycle' subfolder under base_folder, finds all .mat files,
    and loads each one using scipy.io.loadmat.

    Returns:
        A dict mapping each .mat filename (without extension) to the loaded dict.
    """
    drivecycle_dir = Path(base_folder) / "drivecycle"
    if not drivecycle_dir.is_dir():
        raise FileNotFoundError(f"Directory not found: {drivecycle_dir}")

    mat_data = {}
    for mat_file in drivecycle_dir.glob("*.mat"):
        # loadmat returns a dict: variable names → arrays/objects
        try:
            data_dict = sio.loadmat(mat_file)
        except NotImplementedError:
            # If it's a v7.3 MAT-file (HDF5-based), scipy.loadmat will fail.
            # You can either skip it or handle with h5py (see below).
            print(f"Warning: {mat_file.name} might be v7.3. Skipping scipy.loadmat.")
            continue

        # Strip off the path and extension, e.g. "CYC_WLTC_Low_3"
        key = mat_file.stem  
        mat_data[key] = data_dict

        print(f"Loaded '{mat_file.name}' → variables: {[k for k in data_dict.keys() if not k.startswith('__')]}")
    return mat_data

# Load the reference speed
base_folder = ''
all_cycles = load_drivecycle_mat_files(base_folder)

# Example: inspect one of the loaded cycles
example_key = next(iter(all_cycles))
example_data = all_cycles[example_key]

print("\nExample contents of", example_key)
for var_name, arr in example_data.items():
    if var_name.startswith("__"):
        continue
    print(f"  • {var_name}: shape = {getattr(arr, 'shape', 'scalar')}  dtype = {getattr(arr, 'dtype', '')}")


Loaded 'CYC_WLTC_Low_3.mat' → variables: ['cyc_mph']
Loaded 'CYC_WLTP.mat' → variables: ['cyc_mph']
Loaded 'CYC_WLTC_High_3_2.mat' → variables: ['cyc_mph']
Loaded 'CYC_WLTC_Medium_3_2.mat' → variables: ['cyc_mph']
Loaded 'CYC_WLTC_ExtraHigh_3.mat' → variables: ['cyc_mph']

Example contents of CYC_WLTC_Low_3
  • cyc_mph: shape = (590, 2)  dtype = float64


In [ ]:
import Jetson.GPIO as GPIO
GPIO.cleanup()
import time
import board
import busio
from adafruit_pca9685 import PCA9685
import time
import cantools
import can

def controller(lastest_speed, speed_history, reference_speed):

    

    return pwm_output


In [ ]:
#!/usr/bin/env python3
import Jetson.GPIO as GPIO
GPIO.cleanup()
import time
import board
import busio
from adafruit_pca9685 import PCA9685
import time
import cantools
import can

# Load the reference speed
base_folder = os.path.dirname(os.path.abspath(__file__))
all_cycles = load_drivecycle_mat_files(base_folder)

# Example: inspect one of the loaded cycles
example_key = next(iter(all_cycles))
example_data = all_cycles[example_key]

print("\nExample contents of", example_key)
for var_name, arr in example_data.items():
    if var_name.startswith("__"):
        continue
    print(f"  • {var_name}: shape = {getattr(arr, 'shape', 'scalar')}  dtype = {getattr(arr, 'dtype', '')}")

# PWM signal generation setups
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c, address=0x40)
pca.frequency = 1000 #can go up to ~1.6 kHz if you want faster switching
# convert duty cycle percentage → 16-bit value
def set_duty(channel, percent):
    """
    channel: 0–15
    percent: 0.0 to 100.0 (percentage of full on)
    """
    if not (0 <= percent <= 100):
        raise ValueError("percent must be between 0.0 and 100.0")
    # PCA9685 uses 12-bit internally but busdevice scales it to 16-bit.
    # 0 = always off, 0xFFFF = always on. Linear interpolation for duty.
    duty_16bit = int(percent * 65535 / 100)
    pca.channels[channel].duty_cycle = duty_16bit

# Read CAN signal setups
# ─── STEP 1: Load the DBC file ────────────────────────────────────────────────
DBC_PATH = '/home/guiliang/Desktop/DriveRobot/KAVL_V3.dbc'

try:
    db = cantools.database.load_file(DBC_PATH)
except FileNotFoundError:
    print(f"ERROR: Cannot find DBC at '{DBC_PATH}'.")
    raise
# Grab the message definition for "Speed_and_Force" (ID = 0x009)
try:
    speed_force_msg = db.get_message_by_name('Speed_and_Force')
except KeyError:
    print("ERROR: 'Speed_and_Force' not found in the DBC.")
    raise
# ─── STEP 2: Prepare history container ───────────────────────────────────────
force_history = [] 
latest_force = None
speed_history = []
lastest_speed = None
CAN_INTERFACE = 'can0'

# ─── STEP 3: Open the SocketCAN interface ─────────────────────────────────────
# Make sure you've already run (or something similar) on the Jetson:
#   sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on
try:
    bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')
except OSError:
    print(f"ERROR: Cannot open CAN interface '{CAN_INTERFACE}'.")
    print("Make sure you have run something like:\n"
          "    sudo ip link set can0 up type can bitrate 500000 dbitrate 1000000 fd on")
    raise
print(f"Listening on {CAN_INTERFACE} for ID=0x{speed_force_msg.frame_id:03X} ('Speed_and_Force')…")
print("Press Ctrl+C to exit.\n")

# ─── STEP 4: Loop forever, read Force_N, print & store ────────────────────────
try:
    while True:
        # Wait up to 1 second for a CAN frame; msg will be a can.Message or None.
        msg = bus.recv(timeout=1.0)
        if msg is None:
            continue
        # Only decode if the ID matches 0x009:
        if msg.arbitration_id != speed_force_msg.frame_id:
            continue
        # Decode raw payload into a dict {signal_name: value}
        try:
            decoded = db.decode_message(msg.arbitration_id, msg.data)
        except KeyError:
            # ID wasn’t in the DBC (unlikely here), so skip.
            continue
        # Pull out the Force_N field (in Newtons)
        force_value = decoded.get('Force_N')
        speed_value = decoded.get('Speed_kph')
        if force_value is None:
            continue
        if speed_value is None:
            continue
        # Update our “live” variable:
        latest_force = force_value
        lastest_speed = speed_value

        # Append to history list:
        force_history.append(force_value)
        speed_history.append(speed_value)

        # Print live to console
        timestamp = time.time()
        print(f"[{timestamp:.3f}] Force_N = {latest_force:.3f} N; Speed_kph = {lastest_speed:.3f} kph")

        pwm_output = controller(lastest_speed, speed_history, reference_speed)

        if pwm_output >= 0:
            set_duty(0, pwm_output) # channel 0 for accleration pedal
        else:
            set_duty(4, -pwm_output) # channel 4 for break pedal
        print(f"Channel 0 Acceleration dutyCycle: {pwm_output}%; Channel 4 Decceleration dutyCycle: {pwm_output}%;")
        


except KeyboardInterrupt:
    print("\nInterrupted by user. Exiting…")
finally:
    bus.shutdown()

for ch in range(16):
    pca.channels[ch].duty_cycle = 0
pca.deinit()

/tmp/ipykernel_7926/3443721768.py:37: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='socketcan')


Listening on can0 for ID=0x009 ('Speed_and_Force')…
Press Ctrl+C to exit.


Interrupted by user. Exiting…


In [1]:
#!/usr/bin/env python3
import time
import threading
import numpy as np
from smbus2 import SMBus
import cantools
import can

from deepc.utils_deepc import init_pca9685, set_duty_cycle

# ────────── Globals ──────────
latest_speed = None
dyno_can_running = True
CP2112_BUS = 13

# ───────── CAN Listener (reads 'Speed_and_Force') ─────────
def dyno_can_listener_thread(dbc_path: str, can_iface: str):
    global latest_speed, dyno_can_running
    db = cantools.database.load_file(dbc_path)
    msg_def = db.get_message_by_name('Speed_and_Force')
    bus = can.interface.Bus(channel=can_iface, interface='socketcan')
    while dyno_can_running:
        msg = bus.recv(timeout=1.0)
        if msg and msg.arbitration_id == msg_def.frame_id:
            decoded = db.decode_message(msg.arbitration_id, msg.data)
            s = decoded.get('Speed_kph', 0.0)
            latest_speed = float(s)
    bus.shutdown()

# ───────── Relay Auto-Tuner ─────────
class RelayAutoTuner:
    def __init__(self, apply_control, measure_output, relay_amp=10.0):
        self.apply_control = apply_control
        self.measure_output = measure_output
        self.relay_amp = relay_amp

    def tune(self, duration=10.0, dt=0.05):
        t0 = time.time()
        toggle = True
        data_t, data_y = [], []
        next_toggle = t0
        while time.time() - t0 < duration:
            now = time.time()
            if now >= next_toggle:
                u = self.relay_amp if toggle else -self.relay_amp
                self.apply_control(u)
                toggle = not toggle
                next_toggle += dt
            y = self.measure_output()
            data_t.append(now - t0)
            data_y.append(y)
            time.sleep(dt)

        # Estimate Tu from zero crossings
        zs = [data_t[i] for i in range(1, len(data_y))
              if data_y[i-1] * data_y[i] < 0]
        Tu = 2 * np.mean(np.diff(zs))
        amp = (max(data_y) - min(data_y)) / 2
        Ku = (4 * self.relay_amp) / (np.pi * amp)

        # Ziegler–Nichols PID tuning
        Kp = 0.6 * Ku
        Ki = Kp / (0.5 * Tu)
        Kd = 0.125 * Kp * Tu
        return Kp, Ki, Kd

# ───────── Gain-Scheduled PID ─────────
class GainSchedulerPID:
    def __init__(self, apply_control, measure_output):
        self.schedule = {}  # map setpoint → (Kp, Ki, Kd, Kff)
        self.apply_control = apply_control
        self.measure_output = measure_output

    def add_tuned_point(self, setpoint, tuner, **kwargs):
        Kp, Ki, Kd = tuner.tune(**kwargs)
        self.schedule[setpoint] = (Kp, Ki, Kd, 0.0)

    def get_gains(self, sp):
        keys = sorted(self.schedule.keys())
        if sp <= keys[0]:
            return self.schedule[keys[0]]
        if sp >= keys[-1]:
            return self.schedule[keys[-1]]
        for i in range(len(keys)-1):
            k0, k1 = keys[i], keys[i+1]
            if k0 <= sp <= k1:
                g0 = np.array(self.schedule[k0])
                g1 = np.array(self.schedule[k1])
                α = (sp - k0) / (k1 - k0)
                return tuple(((1-α)*g0 + α*g1).tolist())

    def control(self, setpoint, prev_error, integral, dt):
        pv = self.measure_output()
        error = setpoint - pv
        Kp, Ki, Kd, Kff = self.get_gains(setpoint)
        integral += error * dt
        derivative = (error - prev_error) / dt
        u = Kp*error + Ki*integral + Kd*derivative + Kff*(setpoint - pv)
        return u, error, integral

# ───────── Main Application ─────────
def main():
    global dyno_can_running

    # 1) Initialize PCA9685 PWM via CP2112
    bus = SMBus(CP2112_BUS)
    init_pca9685(bus, freq_hz=1000.0)

    # 2) Start CAN thread
    dbc = '/path/to/KAVL_V3.dbc'
    can_thread = threading.Thread(
        target=dyno_can_listener_thread,
        args=(dbc, 'can0'),
        daemon=True
    )
    can_thread.start()

    # 3) PWM helper
    def apply_pwm(u):
        u = float(np.clip(u, -15.0, 100.0))
        if u >= 0:
            set_duty_cycle(bus, 4, 0.0)
            set_duty_cycle(bus, 0, u)
        else:
            set_duty_cycle(bus, 0, 0.0)
            set_duty_cycle(bus, 4, -u)

    # 4) Speed measurement helper
    def measure_speed():
        return latest_speed or 0.0

    # 5) Build tuner and gain scheduler
    relay_tuner = RelayAutoTuner(apply_pwm, measure_speed, relay_amp=10.0)
    pid_sched   = GainSchedulerPID(apply_pwm, measure_speed)

    # 6) Offline tuning at key speeds (kph)
    for sp in [0.0, 20.0, 60.0]:
        input(f"Press Enter when vehicle holds ~{sp} kph...")
        Kp, Ki, Kd = relay_tuner.tune(duration=12.0, dt=0.1)
        pid_sched.schedule[sp] = (Kp, Ki, Kd, 0.0)
        print(f"Tuned @ {sp} kph → Kp={Kp:.2f}, Ki={Ki:.3f}, Kd={Kd:.3f}")

    # 7) Real-time gain-scheduled PID loop
    prev_e, integ = 0.0, 0.0
    Ts = 0.01  # 100 Hz
    try:
        print("Starting gain-scheduled PID. Ctrl+C to stop.")
        while True:
            setpoint = 30.0  # or replace with drive-cycle interpolation
            u, prev_e, integ = pid_sched.control(setpoint, prev_e, integ, Ts)
            apply_pwm(u)
            time.sleep(Ts)

    except KeyboardInterrupt:
        print("Shutting down...")

    finally:
        dyno_can_running = False
        can_thread.join(timeout=1.0)
        # zero all PWM outputs
        for ch in range(16):
            set_duty_cycle(bus, ch, 0.0)
        bus.close()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'deepc'